## Feature Engineering using sklearn

In [1]:
import pickle
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

In [2]:
# load the trained model, scaler and the onehot encoder
model = load_model('model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encoder_geo = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


E0000 00:00:1754847673.986165    5969 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1754847673.987090    5969 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 10000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

/home/bionic/code/nlp-tutorial/env/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [6]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,10000,2,1,1,50000


In [9]:
# encode categorical variables

input_df['Gender'] = label_encoder_gender.transform([input_data['Gender']])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,10000,2,1,1,50000


In [10]:
# concatenate the encoded geography and
# the input data
input_df = pd.concat([input_df, geo_encoder_df], axis=1)
input_df = input_df.drop(columns=['Geography'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,10000,2,1,1,50000,1.0,0.0,0.0


In [11]:
# scalling the input data
input_df = scaler.transform(input_df)

In [13]:
# predict the churn probability
predicted_probability = model.predict(input_df)
predicted_probability

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


array([[0.01436815]], dtype=float32)

In [14]:
predicted_probability[0][0]

0.014368154

In [15]:
if predicted_probability[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
